In [6]:
import tensorflow as tf
import pandas as pd
import random as rd
import numpy as np
import math as m
import sklearn.cluster as skc
import sklearn.neighbors as skn
import sklearn.feature_selection as skf
import matplotlib.pyplot as plt
import scipy as sp
from keras import layers, models

In [4]:
data = pd.read_csv('../Datasets/data.csv', header=0)

In [6]:
cor_count = [0 for i in range(28)]
for i in range(1, 29):
    for j in range(i + 1, 29):
        corr = sp.stats.pearsonr(data.iloc[:, i], data.iloc[:, j])
        if corr[0] > 0.5:
            print(f"{i}, {j}")
            cor_count[i] += 1
            cor_count[j] += 1
            print(corr)
print(cor_count)

1, 2
PearsonRResult(statistic=0.9661792458636241, pvalue=0.0)
3, 5
PearsonRResult(statistic=0.6155235278339611, pvalue=2.5742877301189425e-105)
3, 6
PearsonRResult(statistic=0.5174820482766862, pvalue=1.383799007870868e-69)
3, 7
PearsonRResult(statistic=0.5979100297990985, pvalue=5.6853731657918964e-98)
3, 8
PearsonRResult(statistic=0.5547349176520501, pvalue=9.00388068350439e-82)
3, 9
PearsonRResult(statistic=0.6592941508651425, pvalue=9.589735989744386e-126)
3, 16
PearsonRResult(statistic=0.5098291874461499, pvalue=2.8971450046328056e-67)
3, 20
PearsonRResult(statistic=0.5166204085947175, pvalue=2.543006328414578e-69)
4, 5
PearsonRResult(statistic=0.5099843994039335, pvalue=2.6030556931012904e-67)
4, 6
PearsonRResult(statistic=0.5297678741112126, pvalue=1.94559662256297e-73)
4, 7
PearsonRResult(statistic=0.5377181950481765, pvalue=5.1362150035564535e-76)
4, 9
PearsonRResult(statistic=0.8072597848690927, pvalue=8.902049707846689e-231)
5, 6
PearsonRResult(statistic=0.9044381706706398, 

In [64]:
#Split the raw data into labels and features for the testing & training sets
rawData = data.values.tolist()

#Shuffle the data so it's random
rd.shuffle(rawData)

#Get the labels and relevant features
allLabels = [None] * len(rawData)
allFeatures = [None] * len(rawData)

#Label is column #30, index 29
for i in range(len(allLabels)):
    allLabels[i] = rawData[i][29]

#Need to convert them to ints
for i in range(len(allLabels)):
    label = allLabels[i]
    if(label == "blues"):
        allLabels[i] = 0
    elif(label == "classical"):
        allLabels[i] = 1
    elif(label == "country"):
        allLabels[i] = 2
    elif(label == "disco"):
        allLabels[i] = 3
    elif(label == "hiphop"):
        allLabels[i] = 4
    elif(label == "jazz"):
        allLabels[i] = 5
    elif(label == "metal"):
        allLabels[i] = 6
    elif(label == "pop"):
        allLabels[i] = 7
    elif(label == "reggae"):
        allLabels[i] = 8
    elif(label == "rock"):
        allLabels[i] = 9

#Make them the right dimensions
allLabels = tf.keras.utils.to_categorical(allLabels)

#We want the features at 3, 4, 5, 6, 7, 10, 11, 13, 15, & 17
for i in range(len(allFeatures)):
    rawItem = rawData[i][:]
    item = rawItem[3:8]
    item.append(rawItem[10])
    item.append(rawItem[11])
    item.append(rawItem[13])
    item.append(rawItem[15])
    item.append(rawItem[17])
    allFeatures[i] = item

#Make it the right type
#allFeatures = np.expand_dims(allFeatures, axis=-1)

#Determine how many elements the test set should contain
testSize = round(len(rawData) * 0.2)

#Split the sets
testFeatures = np.asarray(allFeatures[:testSize])
testLabels = np.asarray(allLabels[:testSize])
trainFeatures = np.asarray(allFeatures[testSize:])
trainLabels = np.asarray(allLabels[testSize:])

In [74]:
# Select Data
X = np.array(data.iloc[:, 1:29])
y = np.array(data.iloc[:, 29])
# Make the feature selection model with ANOVA F Measure
fs = skf.SelectKBest(score_func=skf.f_classif, k=10)
# Run the data through selection to obtain final set
X_sel = fs.fit_transform(X, y)
for i in range(len(X_sel[0])):
    print(X_sel[0][i])

#MLP
mlpModel = models.Sequential()
mlpModel.add(layers.Dense(20, activation='relu', input_shape = (10,)))
mlpModel.add(layers.Dense(60))
mlpModel.add(layers.Dense(120))
mlpModel.add(layers.Dense(30))
mlpModel.add(layers.Dense(10))
mlpModel.compile(
    optimizer='adam',
    loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)
#mlpModel.build()
#mlpModel.summary()
history = mlpModel.fit(trainFeatures, trainLabels, epochs=10, shuffle=True, validation_data=(testFeatures, testLabels))

Epoch 1/10
25/25 [==============================] - 7s 269ms/step - loss: 5169.1719 - accuracy: 0.1187 - val_loss: 92.2061 - val_accuracy: 0.1600
Epoch 2/10
25/25 [==============================] - 8s 303ms/step - loss: 15.9740 - accuracy: 0.1275 - val_loss: 2.2845 - val_accuracy: 0.0800
Epoch 3/10
25/25 [==============================] - 8s 337ms/step - loss: 2.2846 - accuracy: 0.1000 - val_loss: 2.2644 - val_accuracy: 0.1150
Epoch 4/10
25/25 [==============================] - 8s 317ms/step - loss: 2.2317 - accuracy: 0.1213 - val_loss: 2.1687 - val_accuracy: 0.1500
Epoch 5/10
25/25 [==============================] - 8s 300ms/step - loss: 2.1446 - accuracy: 0.1725 - val_loss: 2.1856 - val_accuracy: 0.1000
Epoch 6/10
25/25 [==============================] - 8s 310ms/step - loss: 2.1178 - accuracy: 0.1937 - val_loss: 2.1134 - val_accuracy: 0.1200
Epoch 7/10
 1/25 [>.............................] - ETA: 7s - loss: 2.0280 - accuracy: 0.1250

KeyboardInterrupt: 